<a href="https://colab.research.google.com/github/AKMADOU/Assigment_NLP_ADOU_Mathurin/blob/main/logistic_regression_ADOU_Kouam%C3%A9_Mathurin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [3]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]
        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [5]:
def softmax(x):
    max_x = np.max(x)
    exp_x = np.exp(x - max_x)
    sum_exp_x = np.sum(exp_x)
    sm_x = exp_x/sum_exp_x
    return sm_x

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [18]:
def sgd(w, data, niter):
    # get the number of samples in the training
    nlabels, dim = w.shape
    for i in range(niter):
      loss=0
      for y,x in data:
        y_i=np.zeros((10,1))
        y_i[y]=1.0
        x=x.reshape(len(x),1)
        
        # get the softmax of z
        pred = softmax(w@x)
        loss+=-np.log(pred)
        loss_grad=-(y_i-pred)@x.T
        w=w-loss_grad
       
    
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [19]:
from numpy.core.fromnumeric import argmax
def predict(w, x):
   y_pred = softmax(w@x)
   return argmax(y_pred)
    ## FILL CODE

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [22]:
def compute_accuracy(w, valid_data):
    pred_true=0
    for i in range(len(valid_data)):
      x_val=valid_data[i][1]
      y_val=valid_data[i][0]
      index_pre=predict(w, x_val)
      if index_pre==y_val:
        pred_true+=1
        acc=pred_true/len(valid_data)
    return acc*100    

   

In [21]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 92.600

